In [ ]:
import numpy as np
from numba import njit
from timeit import default_timer as timer
from matplotlib import pyplot as plt
import re

## Extracting Energy for Energy vs time graph

In [ ]:
file_path = 'B3LYP_6-31G_5000_0.24.txt'
target_string = 'ETot'  

In [ ]:
def extract_line_with_string(file_path, target_string):
    Etot_line=[]
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if target_string in line:
                Etot_line.append(line)
        return Etot_line

In [ ]:
extracted_line = extract_line_with_string(file_path, target_string)
print(f"Extracted line {target_string}: {extracted_line}")

In [ ]:
def extract_etot_values(lines):
    etot_values = []
    pattern = r'ETot\s*=\s*(-?\d+\.\d+)'
    for line in lines:
        etot_match = re.search(pattern, line)
        if etot_match:
            etot_values.append(float(etot_match.group(1)))
    return etot_values

In [ ]:
etot_array = np.array(extract_etot_values(extracted_line))
type(etot_array)
etot_array.shape
print(etot_array)

In [ ]:
max(etot_array)
j=0
for i in etot_array:
    if i == -343.3490922:
        print(j)
    else: j +=1
        

In [ ]:
def second_largest(numbers):
    sec = []
    sorted_numbers = sorted(numbers, reverse=True)
    for i in sorted_numbers:
        if i < -343.3:
            sec.append(i)
    return sec

In [ ]:
sec_larg = second_largest(etot_array[1100:])
max(sec_larg)

In [ ]:
max_steps = len(etot_array)
max_steps
dt=0.1
time = max_steps*dt

In [ ]:
x = np.arange(0, time, 0.1, dtype=float)
y = etot_array

plt.figure()
plt.scatter(x,y,s=15)
plt.ticklabel_format(style='plain',useMathText=True)
plt.title("Relation between Potential Energy and time")
plt.xlabel("Time (fs)")
plt.ylabel("Potential Energy (Hartree)")

plt.legend()

plt.show()

## Extractin input orientation for the .xyz file for VMD

In [ ]:
start_string = "Input orientation:"
start_line = 0

In [ ]:
def extract_table_from_file(file_path, start_string, start_line, num_lines=12,skipped_lines = 4):
    table_data = []
    is_table_started = False
    i = 0
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for line in lines[start_line:]:
                i+=1
                if start_string in line:
                    is_table_started = True
                    continue  # Skip the line containing the start string
                elif is_table_started and skipped_lines > 0:
                    skipped_lines -=1
                    continue
                elif is_table_started and line.strip() and num_lines > 0 and skipped_lines ==0:
                    table_data.append(line.strip().split())  # Assuming the table is space-separated
                    num_lines -= 1
                elif num_lines == 0:
                    break  # Stop extracting lines once num_lines becomes 0
                    
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    
    return table_data, i


In [ ]:
table_data = extract_table_from_file(file_path, start_string, start_line)
each_table = table_data[0]
current_line = table_data[1]

if table_data:
    print("Extracted Table:")
    for row in table_data:
        print(row)
else:
    print("Table data extraction failed.")

## Extracting Bond Distance between the molecules

In [ ]:
#extracting bond distance
def extract_bond_distance(file_path, start_string, start_line, num_lines=16,skipped_lines = 1):
    table_data = []
    is_table_started = False
    i = 0
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for line in lines[start_line:]:
                i+=1
                if start_string in line:
                    is_table_started = True
                    continue  # Skip the line containing the start string
                elif is_table_started and skipped_lines > 0:
                    skipped_lines -=1
                    continue
                elif is_table_started and line.strip() and num_lines > 0 and skipped_lines ==0:
                    table_data.append(line.strip().split())  # Assuming the table is space-separated
                    num_lines -= 1
                elif num_lines == 0:
                    break  # Stop extracting lines once num_lines becomes 0
                    
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    
    return table_data, i

In [ ]:
start_line_bd = 0
start_string_bd = "Distance matrix (angstroms):"
max_steps = 5001

In [ ]:
bd = extract_bond_distance(file_path, start_string_bd, start_line_bd)

In [ ]:
big_table = []
bd_table = []
i=0
current_line = 0
current_line_bd = 0
while i < max_steps:
    
    table_data = extract_table_from_file(file_path, start_string, current_line)
    big_table.append(table_data[0])
    bd = extract_bond_distance(file_path, start_string_bd, current_line_bd)
    bd_table.append(bd[0])
    #print("total lines of the table",table_data[1])
    current_line += table_data[1]
    current_line_bd += bd[1]
    #print("current line",current_line)
    i+=1
    print(i)
    

In [ ]:
len(big_table)
processed_data = [[[i2 for i2 in item[1:]] for item in sublist] for sublist in big_table]
len(processed_data)


converted_data = [['C' if i[0] == '6' else 'H' if i[0] == '1' else 'O' for i in sublist] 
 for sublist in processed_data]

converted_data[0][1]
processed_data

In [ ]:
# Output file path
output_file_path = 'B3LYP_6-31G_5000_0.24.xyz'

# Write data to XYZ file
with open(output_file_path, 'w') as file:
    i=0
    num_atoms = 12
    j = 0
    for table in processed_data:
        file.write(f"{num_atoms}\n")
        file.write("step "+ str(j)+ "\n")
        for atoms in table:
            
            atom_label = converted_data[0][i]
            x, y, z = map(float, atoms[2:])
            file.write(f"{atom_label} {x} {y} {z}\n")
            i+=1
        i = 0
        j+=1

In [ ]:
def extract_spec_bond(input_data, target_A, target_B):
    wanted_BD = []
    for table in input_data:
        line = table[target_A-1]
        wanted_BD.append(line[target_B+1])
    return wanted_BD

In [ ]:
print(len(bd_table))
wanted = np.array(extract_spec_bond(bd_table,2,1))
bd_table

In [ ]:
type(wanted)
print(wanted[0:100])

In [ ]:
#oxygen= O7, hydrogen= H8
def extract_OH_bond(input_data, target_A, target_B):
    wanted_BD = []
    for table in input_data:
        line = table[target_A+7]
        wanted_BD.append(line[target_B-4])
    return wanted_BD

In [ ]:
#C1C2
wanted_float = wanted.astype(float)
x = np.arange(0, time, 0.1, dtype=float)
# Create a scatter plot
plt.scatter(x, wanted_float, s=1)

# Set axis labels

plt.xlabel("Time (fs)")
plt.ylabel('Bond Distance (angstrom)')

# Show the plot
plt.show()

In [ ]:
O7H8 = np.array(extract_OH_bond(bd_table, 8, 7))
x = np.arange(0, time, 0.1, dtype=float)
plt.scatter(x, O7H8.astype(float), s = 1)


# Set axis labels

plt.xlabel("Time (fs)")
plt.ylabel('Bond Distance (angstrom)')

# Show the plot
plt.show()

In [ ]:
C3C2 = np.array(extract_spec_bond(bd_table,3,2))
O4C3 = np.array(extract_spec_bond(bd_table, 4, 3))
O7C2 = np.array(extract_spec_bond(bd_table, 7, 2))
O6C3 = np.array(extract_spec_bond(bd_table, 6, 3))

# Create a scatter plot
x = np.arange(0, time, 0.1, dtype=float)
#plt.scatter(x, O4C3.astype(float), color = 'red',label = 'O4C3', s = 1)
#plt.scatter(x, C3C2.astype(float), color = 'purple',label = 'C3C2', s = 1)
plt.scatter(x, O7H8.astype(float),label = 'O7H8', s = 1)

plt.legend()

# Set axis labels
plt.xlabel("Time (fs)")
plt.ylabel('Bond Distance (angstrom)')

# Show the plot
plt.show()

In [ ]:
def read_file(file_path):
    # Initialize empty lists to store x and y values
    x_values = []
    y_values = []

    # Read data from the text file
    with open(file_path, 'r') as file:

        for line in file:
            # Assuming columns are separated by whitespace, you may need to adjust accordingly
            columns = line.split()

            # Convert the values to float
            x_values.append(float(columns[0]))
            y_values.append(float(columns[1]))
    return x_values, y_values

In [ ]:

# Specify the file path
file_path = 'data996.txt'  # Replace 'your_file.txt' with the actual path to your text file
r1 = read_file(file_path)
file_2 = 'data682.txt'  # Replace 'your_file.txt' with the actual path to your text file
r2 = read_file(file_2)

# Plot the data
plt.plot(r1[0], r1[1], marker='o',label='step 996', linestyle='-', color='b', markersize=1)
plt.plot(r2[0], r2[1], marker='o',label='step 682', linestyle='-', color='purple', markersize=1)
plt.xlabel('steps')
plt.ylabel('Energy (hatree)')
plt.ylim(-343.65,-343.30)
plt.title('Transition state optimization')
plt.xlim(0,30)
plt.grid(True)
plt.legend()
plt.show()

y = max(y_values[10:])-y_values[12]
y